# Hitter Data Cleaning
The purpose of this notebook is to clean the hitter scouting report data, as found in scouting_report_scraping.ipynb.

## Setup/Initiation:

First, importing needed packages:

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 20,10
import numpy as np
import glob
from scipy import stats
from bs4 import BeautifulSoup
import requests
import re
from IPython.core.display import display, HTML # make sure Jupyter knows to display it as HTML

Bringing in the pickled data:

In [2]:
pwd

'/Users/patrickbovard/Documents/GitHub/MLB-Scouting-Reports-Analysis'

In [3]:
with open('./Data_Files/initial_hitter_data.pickle', 'rb') as read_file:
    initial_hitter_data = pickle.load(read_file)

In [4]:
initial_hitter_data.head()

,Name,Born,Bats,Throws,Height,Weight,Physical_Health,MLB_ETA,Risk_Factor,OFP,...,Power_Grade,Running_Grade,Glove_Grade,Arm_Grade,Hit_Report,Power_Report,Running_Report,Glove_Report,Arm_Report,Overall_Report
0,CJ Abrams,10/03/2000,Left,Right,"6' 2""",185,"A pure athlete with quick twitch muscles, very...",2025,Extreme,50,...,40,70,55,60,Very raw as a hitter with a lot of mechanical ...,Beyond needing to gain strength to add to his ...,"Easily his best tool, runs like a wide receive...","Good hand-eye coordination and fundamentals, c...","On pure arm strength, it's average to slightly...",Every year there are players drafted purely ba...
1,Osvaldo Abreu,06/13/1994,Right,Right,"6' 0""",195,Smaller frame with a little room to add streng...,2018,Moderate,45,...,30,55,55,55,"Open stance, steady at the plate, hands are ti...","Frame translates to below-average raw power, l...","4.15 clock, above-average foot speed speed, qu...","Quick transfer, flashes soft hands, immature g...",Enough arm strength to make any throw from sho...,Abreu has the defensive chops and arm strength...
2,Osvaldo Abreu,06/13/1994,Right,Right,"6' 0""",195,Small frame; toned body with muscular definiti...,2019,High,40,...,30,45,50,50,Above-average bat speed; noisy hands; minor hi...,Average raw power; slight leverage and above-a...,4.45 home to first; slow out of the box; more ...,Extremely inconsistent currently; footwork can...,Above-average arm strength; moderate carry and...,Abreu was signed as an international free agen...
3,Ronald Acuna,12/18/1997,Right,Right,"6' 0""",180,"Quick-twitch movement, explosive hands; presen...",2019,High,70,...,60,55,45,55,"High hands stay quiet through load, attacks ba...",Plus-plus raw power; outsized raw despite aver...,"Times range 4.2-4.3, 4.15 on a leaner out of b...",Uncertain center-field future; above-average r...,Above-average arm for center; would play to av...,Acuna flashes all five tools with quick-twitch...
4,Ronald Acuna,12/18/1997,Right,Right,"6' 0""",180,"Impressive, proportioned athlete with plus str...",2018,Low,80,...,70,55,55,70,High hands (above ear) in set-up. Moderate leg...,Wrist strength and quickness is major asset. G...,4.23 home to first. Faster underway with longe...,Demonstrates good reads and solid first step; ...,One-hopper on line from RF warning track. Thro...,Impressive player ready to contribute now desp...


In [5]:
initial_hitter_data.shape

(524, 21)

All the data is here, 524 batters with 21 hitting columns. 

In [6]:
initial_hitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             524 non-null    object
 1   Born             524 non-null    object
 2   Bats             524 non-null    object
 3   Throws           524 non-null    object
 4   Height           524 non-null    object
 5   Weight           524 non-null    object
 6   Physical_Health  524 non-null    object
 7   MLB_ETA          524 non-null    object
 8   Risk_Factor      524 non-null    object
 9   OFP              524 non-null    object
 10  Hit_Grade        524 non-null    object
 11  Power_Grade      524 non-null    object
 12  Running_Grade    524 non-null    object
 13  Glove_Grade      524 non-null    object
 14  Arm_Grade        524 non-null    object
 15  Hit_Report       524 non-null    object
 16  Power_Report     524 non-null    object
 17  Running_Report   524 non-null    ob

## Initial Cleaning
First up, the following columns will need to have their type changed or adjusted.  This will be done before text cleaning, to prepare the dataframe:
- Born: datetime, used to find age
- Bats/Throws: one hot encode to get to 1/0 for bats/throws right/left (will need another value for switch hitters)
- Height (convert to inches)
- Weight: numerical
- MLB ETA (year, perhaps for age relative to ETA)
- OFP and all other grades: numerical (int)
I'll start by building out functions to tackle these down the list.

### Grades and Weight
This is the simplest, I just need to change these to numerical.  Functions are being written in hitter_cleaning.py for cleanliness of the notebook.

In [7]:
#List of the columns needed:
num_col_list = ['Weight', 'OFP'] #will use if '_Grade' in col for the grade columns, rather than typing out each one

In [8]:
from hitter_cleaning import num_col_maker, grade_num_col_maker, numerical_columns_combined

In [9]:
numerical_df = numerical_columns_combined(initial_hitter_data, num_col_list)

In [10]:
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               524 non-null    object
 1   Born               524 non-null    object
 2   Bats               524 non-null    object
 3   Throws             524 non-null    object
 4   Height             524 non-null    object
 5   Physical_Health    524 non-null    object
 6   MLB_ETA            524 non-null    object
 7   Risk_Factor        524 non-null    object
 8   Hit_Report         524 non-null    object
 9   Power_Report       524 non-null    object
 10  Running_Report     524 non-null    object
 11  Glove_Report       524 non-null    object
 12  Arm_Report         524 non-null    object
 13  Overall_Report     524 non-null    object
 14  Weight_Num         524 non-null    int64 
 15  OFP_Num            524 non-null    int64 
 16  Hit_Grade_Num      524 non-null    int64 
 1

### Bats and Throws
In order to use this for any quantitative purposes, will need to convert the bats and throws columns to numeric values (i.e. 1/0).

In [11]:
numerical_df.Bats.value_counts()

Right     268
Left      184
Switch     71
            1
Name: Bats, dtype: int64

In [12]:
numerical_df.Throws.value_counts()

Right    447
Left      76
           1
Name: Throws, dtype: int64

Before getting in, checking out those blanks:

In [13]:
numerical_df[(numerical_df.Throws != 'Right') & (numerical_df.Throws != 'Left')]

,Name,Born,Bats,Throws,Height,Physical_Health,MLB_ETA,Risk_Factor,Hit_Report,Power_Report,...,Glove_Report,Arm_Report,Overall_Report,Weight_Num,OFP_Num,Hit_Grade_Num,Power_Grade_Num,Running_Grade_Num,Glove_Grade_Num,Arm_Grade_Num
307,NOTUSED NOTUSED,00/00/0000,,,"0' 0""","Strong, compact, athletic body, projection rem...",2019,High,"Quick hands, premium bat speed; varies timing ...","Plus raw, premium bat speed with moderate leve...",...,"Adequate in the field, tracks well, takes good...","Plus arm strength, accurate to third base, car...",Martinez struggled to settle in to his full-se...,0,55,50,50,55,50,60


Will need to remove this row before final analysis, since that isn't usable as I don't have a player name.  Report is [here](https://legacy.baseballprospectus.com/prospects/eyewitness_bat.php?reportid=389).

Back to the processing, I'll use one hot encoding for the following:
- Bats: Bat_Rights and Bat_Left, since switch hitters do both
- Throws: Right will be 1, Left will be 0 --> no one does both

In [14]:
from hitter_cleaning import bats_preprocessing, throws_preprocessing, throws_bats_processor

In [15]:
throws_bats_df = throws_bats_processor(numerical_df)

/Users/patrickbovard/Documents/GitHub/MLB-Scouting-Reports-Analysis/hitter_cleaning.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Bats_Left[i] = 1
/Users/patrickbovard/Documents/GitHub/MLB-Scouting-Reports-Analysis/hitter_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Bats_Right[i] = 1
/Users/patrickbovard/Documents/GitHub/MLB-Scouting-Reports-Analysis/hitter_cleaning.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [16]:
throws_bats_df.Throws.value_counts()

Right    447
Left      76
           1
Name: Throws, dtype: int64

In [17]:
throws_bats_df.Throws_Right.value_counts()

1    447
0     77
Name: Throws_Right, dtype: int64

All Set.

### Height
This is relatively simple, just converting the height column to inches for best computation.

In [18]:
throws_bats_df.Height.value_counts()

6' 0"     105
6' 1"      86
6' 3"      82
6' 2"      72
6' 4"      53
5' 11"     40
5' 10"     35
5' 9"      15
6' 5"      14
5' 8"       7
6' 6"       5
6' 7"       5
5' 7"       2
5' 6"       2
0' 0"       1
Name: Height, dtype: int64

The easiest way to convert this will be to use string splitting, on the ' (feet) and " (inches).

In [19]:
from hitter_cleaning import height_converter_to_inches, height_inches_column

In [20]:
height_df = height_inches_column(throws_bats_df)

In [21]:
height_df['Height_Inches'].value_counts()

72    105
73     86
75     82
74     72
76     53
71     40
70     35
69     15
77     14
68      7
78      5
79      5
66      2
67      2
0       1
Name: Height_Inches, dtype: int64

All set on height.

### MLB ETA and Birthday
Last step for this initial cleaning is to do the MLB ETA and Birthday, in order to calculate age.

**ETA**:

In [22]:
height_df.MLB_ETA.value_counts()

2018          102
2017           86
2019           74
2016           60
2021           49
2020           36
2015           35
2022           25
N/A            10
2014            9
                8
Late 2015       5
Late 2017       4
Late 2016       4
2023            3
Mid 2016        2
2013            2
11/1/2021       1
Sept. 2019      1
2025            1
None            1
Mid 2015        1
mid-2019        1
Late 2020       1
Late-2019       1
2024            1
late 2019       1
Name: MLB_ETA, dtype: int64

There are a lot of different values here, but the main constant is having the year.  Ultimately that is the crucial things, I don't care as much about late, or not.  This will take a few different scenarios, based on the following:
- Easiest: pure year is listed
- Words or word- with year (i.e. Late 2015, or Late-2019) --> for this, can split on space or dash and take the second element.
- Date format: split on /, take the last element
- N/A or blank: list "0" for now

In [23]:
from hitter_cleaning import eta_cleaner, cleaned_eta_column

In [24]:
eta_df = cleaned_eta_column(height_df)

In [25]:
eta_df.ETA_clean.dtypes

dtype('int64')

**Birthday**:  
Birthdays are in the following format: MM/DD/YYYY.  The only thing to really clean here is to convert them to a datetime type.

In [26]:
eta_df.Born.value_counts()

06/11/1993    5
08/17/1993    4
09/08/1993    4
08/04/1993    4
05/27/1996    4
             ..
02/14/1995    1
07/07/1995    1
01/25/1993    1
12/16/1994    1
02/10/1995    1
Name: Born, Length: 386, dtype: int64

In [27]:
from hitter_cleaning import born_cleaner

In [28]:
born_df = born_cleaner(eta_df)

In [29]:
born_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               524 non-null    object        
 1   Born               524 non-null    object        
 2   Bats               524 non-null    object        
 3   Throws             524 non-null    object        
 4   Height             524 non-null    object        
 5   Physical_Health    524 non-null    object        
 6   MLB_ETA            524 non-null    object        
 7   Risk_Factor        524 non-null    object        
 8   Hit_Report         524 non-null    object        
 9   Power_Report       524 non-null    object        
 10  Running_Report     524 non-null    object        
 11  Glove_Report       524 non-null    object        
 12  Arm_Report         524 non-null    object        
 13  Overall_Report     524 non-null    object        
 14  Weight_Num

Utilizing the chained functions together:

In [30]:
from hitter_cleaning import master_hitter_cleaner

In [31]:
cleaned_df = master_hitter_cleaner(initial_hitter_data, drop_original_columns=True)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524 entries, 0 to 523
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               524 non-null    object        
 1   Physical_Health    524 non-null    object        
 2   Risk_Factor        524 non-null    object        
 3   Hit_Report         524 non-null    object        
 4   Power_Report       524 non-null    object        
 5   Running_Report     524 non-null    object        
 6   Glove_Report       524 non-null    object        
 7   Arm_Report         524 non-null    object        
 8   Overall_Report     524 non-null    object        
 9   Weight_Num         524 non-null    int64         
 10  OFP_Num            524 non-null    int64         
 11  Hit_Grade_Num      524 non-null    int64         
 12  Power_Grade_Num    524 non-null    int64         
 13  Running_Grade_Num  524 non-null    int64         
 14  Glove_Grad

From here, I have my cleaned data and can move into text-preprocessing.  Pickling this data out to utilize in a new notebook.

In [32]:
with open('./Data_Files/cleaned_hitter_data.pickle', 'wb') as to_write:
    pickle.dump(cleaned_df, to_write)

# Next: initial_text_processing.ipynb